<a href="https://colab.research.google.com/github/ameymane09/Holidays-analysis-for-marketing/blob/main/Assignment_2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

Having a great marketing strategy in place is key to the success of any business. Without a marketing strategy, you lack focus. And without focus, you will, quite simply, fail to reach any of the goals and objectives that you have set. Any information about customers allows marketers to gain a laser-sharp understanding of their target audience. The marketing budget is being set for the year 2023. The marketing director would like to know which holiday brings in the most money so the team can adjust the marketing dollars. <br><br>
### Objective

What holidays should the marketing team invest more marketing dollars in? Also, find out whatever other insights you can from the dataset. <br><br>

### Instructions

Answer the questions (using SQL, Python) and explain your rationale in the write-up:

1. How would you segment holidays based on the expenditure of customers
2. Which of these segments / sub-segments would you propose be approved?
    - For e.g. Does a certain holiday drive the sales of a particular segment of people and how can the marketing team use that to optimize their plan? Would a holiday season’s duration affect the number of sales and how should the marketing team strategize around that?
3. What other insights in general can you share about these segments?
4. Tell us what your observations were on the data itself (completeness, skews) and how you would treat any anomalies (for eg - missing data)

# Importing, reading and cleaning the data

In [146]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import calendar
pd.options.plotting.backend = "plotly"

In [147]:
ecom_df = pd.read_csv("/content/Ecommerce_Data.csv")
holiday_df = pd.read_csv("/content/US_Holiday_Dates_(2004-2021).csv")

In [148]:
ecom_df.head()
ecom_df.info()
ecom_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81601 entries, 0 to 81600
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   81601 non-null  int64  
 1   InvoiceNo    81601 non-null  object 
 2   StockCode    81601 non-null  object 
 3   Description  81601 non-null  object 
 4   Quantity     81601 non-null  int64  
 5   UnitPrice    81601 non-null  float64
 6   CustomerID   81601 non-null  int64  
 7   Country      81601 non-null  object 
 8   Date         81601 non-null  object 
 9   Hour         81601 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 6.2+ MB


,Unnamed: 0,Quantity,UnitPrice,CustomerID,Hour
count,81601.000000,81601.000000,81601.000000,81601.000000,81601.000000
mean,278271.366772,11.965736,3.168721,15283.816215,12.729783
std,152483.054308,45.782018,18.731668,1713.292081,2.288777
min,2.000000,-3114.000000,0.000000,12347.000000,6.000000
25%,148283.000000,2.000000,1.250000,13949.000000,11.000000
50%,284742.000000,5.000000,1.950000,15144.000000,13.000000
75%,409445.000000,12.000000,3.750000,16790.000000,14.000000
max,541908.000000,3186.000000,4287.630000,18287.000000,20.000000


Since the unnamed column is not conveying any useful info, I've decided to drop it since it is impossible to figure out what it stands for.

In [149]:
ecom_df.drop(labels="Unnamed: 0", axis=1, inplace=True)
ecom_df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice',
       'CustomerID', 'Country', 'Date', 'Hour'],
      dtype='object')

In [150]:
holiday_df.head()
holiday_df.info()
holiday_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     342 non-null    object
 1   Holiday  342 non-null    object
 2   WeekDay  342 non-null    object
 3   Month    342 non-null    int64 
 4   Day      342 non-null    int64 
 5   Year     342 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 16.2+ KB


,Month,Day,Year
count,342.000000,342.000000,342.000000
mean,7.263158,15.853801,2012.500000
std,3.899889,9.653330,5.195729
min,1.000000,1.000000,2004.000000
25%,4.000000,6.000000,2008.000000
50%,9.000000,16.500000,2012.500000
75%,11.000000,24.000000,2017.000000
max,12.000000,31.000000,2021.000000


## Checking for null values

In [151]:
ecom_df.isna().any()

InvoiceNo      False
StockCode      False
Description    False
Quantity       False
UnitPrice      False
CustomerID     False
Country        False
Date           False
Hour           False
dtype: bool

In [152]:
holiday_df.isna().any()

Date       False
Holiday    False
WeekDay    False
Month      False
Day        False
Year       False
dtype: bool

The data contains no null values so we can proceed with operations on data now.

## Fixing the date formatting

In [153]:
ecom_df["Date"] = pd.to_datetime(ecom_df["Date"], format='%Y-%m-%d')
holiday_df["Date"] = pd.to_datetime(holiday_df["Date"], format='%Y-%m-%d')

In [154]:
ecom_df[["Date"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81601 entries, 0 to 81600
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    81601 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 637.6 KB


## ROCCC Analysis

1. Reliability: The data originates from an e-commerce platform and collected internally through various tools and hence, it is highly reliable.
2. Originality: As mentioned above, since it is internal data, we can be confident that it is original and specific to this business.
3. Comprehensive: The data spans a year from 2010 to 2011 but we should have more to concretely identify and distinguish between one-off trends and established ones.
4. Current: The data is more than 11 years old and hence, may not reflect current trends accurately.
5. Cited: No need to be cited as data is internal.

# Solution

###Analysing the distribution of orders

In [155]:
print("Total number of countries in the dataset: ", ecom_df["Country"].nunique())

Total number of countries in the dataset:  37


There seem to be orders from many different countries. It looks like the E-commerce business operates worldwide. Let us find out the distribution of orders by country so we can narrow our analysis to specific regions for a more effective marketing strategy.

In [156]:
orders_by_country = ecom_df.groupby("Country")["Quantity"].count().sort_values(ascending=False)
orders_by_country.head()

Country
United Kingdom    72617
Germany            1898
France             1689
EIRE               1500
Spain               504
Name: Quantity, dtype: int64

In [157]:
orders_graph = px.histogram(x=orders_by_country.index, y=orders_by_country.values,
                            labels=dict(
                                x = "",
                                y = "Number of orders"
                                ))

orders_graph.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Distribution of Orders Worldwide",
                            y = 0.95,
                            x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top'
                      ))
orders_graph.update_xaxes(tickangle=45)
orders_graph.show()

In [281]:
orders_pie = px.pie(orders_by_country, values=orders_by_country.values, 
                    names=orders_by_country.index)
orders_pie.update_traces(textposition='inside', textinfo='percent+label')
orders_pie.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
orders_pie.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Distribution of Orders Worldwide",
                            y = 0.95,
                            x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top'
                      ))
orders_pie.update_layout(legend=dict(
                            yanchor="top",
                            y=0.99,
                            xanchor="left",
                            x=0.01
                        ))
orders_pie.show()

Seems like the E-commerce company operates most of its business in the UK since we can see that most orders are from there itself with a few orders for other countries. <br> <br>
Since most of the presence of this company is based in and around the UK, we will focus our analysis towards them and drop other countries.

In [159]:
top_list = pd.Series.to_list(ecom_df.value_counts("Country").index[:8])
print("The top countries are: ", ", ".join(top_list), "\n\n")

eu_ecom_df = ecom_df[ecom_df["Country"].isin(top_list)]
eu_ecom_df.info()

The top countries are:  United Kingdom, Germany, France, EIRE, Spain, Netherlands, Belgium, Switzerland 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 79414 entries, 0 to 81600
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    79414 non-null  object        
 1   StockCode    79414 non-null  object        
 2   Description  79414 non-null  object        
 3   Quantity     79414 non-null  int64         
 4   UnitPrice    79414 non-null  float64       
 5   CustomerID   79414 non-null  int64         
 6   Country      79414 non-null  object        
 7   Date         79414 non-null  datetime64[ns]
 8   Hour         79414 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 6.1+ MB


###Analysing Trends Related to Customer Spending

We will add a column named `MoneySpent` to show the amount of money spent by the customer for a particular order. <br>`MoneySpent` = `Quantity` * `UnitPrice` <br>This will also help us quantify data regarding customer spending habits.

In [160]:
eu_ecom_df["MoneySpent"] = eu_ecom_df["Quantity"] * eu_ecom_df["UnitPrice"]
eu_ecom_df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,MoneySpent
0,536551,22112,CHOCOLATE HOT WATER BOTTLE,1,4.95,17346,United Kingdom,2010-12-01,14,4.95
1,536412,22900,SET 2 TEA TOWELS I LOVE LONDON,2,2.95,17920,United Kingdom,2010-12-01,11,5.90
2,536562,22313,OFFICE MUG WARMER PINK,6,2.95,13468,United Kingdom,2010-12-01,15,17.70
3,536528,22865,HAND WARMER OWL DESIGN,1,2.10,15525,United Kingdom,2010-12-01,13,2.10
4,536378,21975,PACK OF 60 DINOSAUR CAKE CASES,24,0.55,14688,United Kingdom,2010-12-01,9,13.20


In [161]:
sorted_ecom_df = eu_ecom_df.groupby("Date").agg({"MoneySpent": pd.Series.sum,
                                                 "Quantity": pd.Series.sum})
sorted_ecom_df.reset_index(inplace=True)
sorted_ecom_df.head()

,Date,MoneySpent,Quantity
0,2010-12-01,8025.85,3729
1,2010-12-02,7660.42,7154
2,2010-12-03,3362.06,1535
3,2010-12-05,6620.81,2916
4,2010-12-06,6375.55,3025


In [248]:
# Line Graph
money_spent = go.Figure()
money_spent.add_scatter(x=sorted_ecom_df["Date"], y=sorted_ecom_df["MoneySpent"], 
                        fill='tozeroy', name="MoneySpent")
money_spent.add_scatter(x=sorted_ecom_df["Date"], y=sorted_ecom_df["Quantity"],
                        name="Quantity")

money_spent.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                          text = "Customer Spending Throughout the Year",
                          y = 0.95,
                          x = 0.5,
                          xanchor = 'center',
                          yanchor = 'top'
                          ),
                      legend=dict(
                          orientation = "h", 
                          yanchor = "bottom", 
                          y = 1.02, 
                          xanchor = "right",
                          x = 1))
money_spent.show()

In [163]:
# Box Plot for Money Spent Per Month
money_spent_per_month = px.box(sorted_ecom_df, 
                               x=sorted_ecom_df["Date"].dt.month, 
                               y="MoneySpent", 
                               labels={
                                      "x": ""
                                      })

money_spent_per_month.add_vrect(x0=9, x1=12, 
              annotation_text="Jump in Spending", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)

money_spent_per_month.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Money Spent Per Month",
                            y = 0.95,
                            x = 0.5,
                            xanchor = 'center',
                            yanchor = 'top'
                      ))

money_spent_per_month.update_layout(
                    xaxis = dict(
                        tickmode = 'array',
                        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                        ticktext = list(calendar.month_name)
                    )
                )

money_spent_per_month.show()

In [235]:
# Box Plot for Money Spent Per Week
money_spent_per_week = px.box(sorted_ecom_df, 
                               x=sorted_ecom_df["Date"].dt.isocalendar().week, 
                               y="MoneySpent", 
                               labels=dict(
                                      x = "Week Number"
                                      ))

money_spent_per_week.add_vrect(x0=35, x1=40, 
              annotation_text="Jump in Spending", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)

money_spent_per_week.add_vrect(x0=44, x1=48, 
              annotation_text="Jump in Spending", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)


money_spent_per_week.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Money Spent Per Week",
                            y = 0.95,
                            x = 0.5,
                            xanchor = 'center',
                            yanchor = 'top'),
                      xaxis = dict(
                              tickmode = 'linear',
                              tick0 = 0,
                              dtick = 1
    ))

money_spent_per_week.show()

In [165]:
# Box Plot for Money Spent Per Weekday
money_spent_per_weekday = px.box(sorted_ecom_df, 
                               x=sorted_ecom_df["Date"].dt.weekday, 
                               y="MoneySpent", 
                               labels={
                                      "x": ""
                                      })

money_spent_per_weekday.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Money Spent Per Day of the Week",
                            y = 0.95,
                            x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top'
                      ))

money_spent_per_weekday.update_layout(
                    xaxis = dict(
                        tickmode = 'array',
                        tickvals = [0, 1, 2, 3, 4, 5, 6],
                        ticktext = list(calendar.day_name)
                    ))
money_spent_per_weekday.show()

In [166]:
hourly_stats = eu_ecom_df.groupby("Hour").agg({"MoneySpent": pd.Series.sum,
                                                   "Quantity": pd.Series.sum})
hourly_stats.sort_values("MoneySpent", ascending=False, inplace=True)

hourly_spending = px.histogram(hourly_stats, hourly_stats.index,
                     "MoneySpent", nbins=48, hover_data=hourly_stats.columns,
                     histfunc="avg", text_auto=True)
hourly_spending.update_layout(titlefont=dict(size=20, color='black'),
                   title={
                        'text': "Hourly Customer Spending",
                        'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                   ) 
hourly_spending.show()

In [167]:
# eu_ecom_df[eu_ecom_df["Weekday"] == "Saturday"]

#### Insights:

1. From the graphs above, it is evident that there are spikes in both, number of orders and customer spending as a whole throughout the year. 
2. We can also see that the there is a jump in customer spending from September to December with the highest being in November.
3. Data is missing for Week 52 (last week). This maybe due to the business being shut for the winter holidays.
3. There is no data for any Saturday. It seems like the business is closed on Saturdays but it is quite strange for an e-commerce business.
4. Thursday is most popular while Sunday is the least.
5. Most active spending hours are from  10 AM to 3 PM.

### Returns

From the graphs above, we can see that the `Quantity` data contains some negative values. I'm assuming this means that the particular item was returned. <br>
There are also some days where the `MoneySpent` is in negative (which means people have returned items worth more that day than they have purchased in total).

It will be interesting to see why such incidents happen where people return more items than they buy but that is out of our scope for now.

In [168]:
eu_ecom_df[eu_ecom_df["MoneySpent"] < 0]

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,MoneySpent
96,C536548,22168,ORGANISER WOOD ANTIQUE WHITE,-2,8.50,12472,Germany,2010-12-01,14,-17.00
194,C536548,22077,6 RIBBONS RUSTIC CHARM,-6,1.65,12472,Germany,2010-12-01,14,-9.90
230,C536548,20914,SET/5 RED RETROSPOT LID GLASS BOWLS,-1,2.95,12472,Germany,2010-12-01,14,-2.95
231,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,0.29,17548,United Kingdom,2010-12-01,10,-6.96
266,C536548,22654,DELUXE SEWING KIT,-1,5.95,12472,Germany,2010-12-01,14,-5.95
...,...,...,...,...,...,...,...,...,...,...
81315,C581330,22959,WRAP CHRISTMAS VILLAGE,-25,0.42,15877,United Kingdom,2011-12-08,11,-10.50
81366,C581468,22098,BOUDOIR SQUARE TISSUE BOX,-12,0.39,13599,United Kingdom,2011-12-08,19,-4.68
81382,C581316,21531,RED RETROSPOT SUGAR JAM BOWL,-1,2.55,12523,France,2011-12-08,11,-2.55
81398,C581465,22171,3 HOOK PHOTO SHELF ANTIQUE WHITE,-1,8.50,15755,United Kingdom,2011-12-08,18,-8.50


### Working Around Missing Data

We will merge this data with the holidays column to check what holidays have had most spending.

In [169]:
df_merged = holiday_df.merge(sorted_ecom_df, how="inner").sort_values("MoneySpent", ascending=False)
df_merged.head(20)

,Date,Holiday,WeekDay,Month,Day,Year,MoneySpent,Quantity
7,2011-11-23,Thanksgiving Eve,Wednesday,11,23,2011,13208.58,7448
1,2011-10-10,Columbus Day,Monday,10,10,2011,8524.41,4666
9,2011-11-11,Veterans Day,Friday,11,11,2011,6948.25,4191
0,2011-07-04,4th of July,Monday,7,4,2011,6776.75,2346
3,2011-09-05,Labor Day,Monday,9,5,2011,6631.32,4090
6,2011-11-24,Thanksgiving Day,Thursday,11,24,2011,6011.60,3501
10,2011-02-21,Washington's Birthday,Monday,2,21,2011,5365.04,3615
8,2011-02-14,Valentine’s Day,Monday,2,14,2011,4681.92,3414
2,2011-06-19,Juneteenth,Sunday,6,19,2011,4431.40,2812
4,2011-09-04,Labor Day Weekend,Sunday,9,4,2011,3636.26,2799


We can see that `df_merged` is missing Christmas data. There is no data for 24th and 25th of December 2010 but we can't ignore christmas as it is one of the most important holidays of the year.

In [170]:
sorted_ecom_df[(sorted_ecom_df["Date"] >= "2010-12-24") & 
               (sorted_ecom_df["Date"] <= "2010-12-25")]

,Date,MoneySpent,Quantity


In [171]:
sorted_ecom_df[sorted_ecom_df["Date"].dt.year == 2011].head(5)

,Date,MoneySpent,Quantity
20,2011-01-04,2107.40,1381
21,2011-01-05,4905.26,4007
22,2011-01-06,4916.99,2543
23,2011-01-07,4250.22,2523
24,2011-01-09,2772.11,1361


Upon further inspection, there is data missing from 24th December 2010 to 3rd January 2011. We will have to adjust accordingly and add the missing data manually.

##Understanding the Data as a Whole

Now let us try to visualize customer spending around the holidays. I have highlighted the holidays in a different colour so as to make it easy to spot them.

In [218]:
color_discrete_sequence = ['#00ABB3']*len(sorted_ecom_df)
sorted_ecom_df['Category'] = [str(i) for i in sorted_ecom_df.index]

# If date is a holiday, mark it in a different colour
bar_counter = 0
for date in sorted_ecom_df.Date:
  for hol_date in holiday_df.Date:
    if date == hol_date:
      color_discrete_sequence[bar_counter] = "#CF0A0A"
  bar_counter += 1

spending_vs_holidays = px.bar(data_frame=sorted_ecom_df,
              x="Date", 
              y="MoneySpent", 
              color=sorted_ecom_df['Category'], 
              color_discrete_sequence=color_discrete_sequence)

spending_vs_holidays.update_layout(showlegend=False,
                      titlefont=dict(size=20, color='black'),
                      title=dict(
                            text = "Customer Spending Around the Holidays",
                            y = 0.95,
                            x = 0.5,
                            xanchor =  'center',
                            yanchor = 'top'
                      ))
spending_vs_holidays.show()

It is clear from the above graph that spending is definitely influenced by holidays but not on the exact date, rather some time before that. It makes sense since people tend to buy gifts in advance and are also aware of the shipping delays. <br><br>
Insights:


1.   September to December is the busy season as a lot of spending is concentrated there.
2.   There is a spike in spending before most holidays (some more than others).
Especially on holidays where gifting items is a tradtion, we see a massive spike in customer spending. For example, just before Valentine's Day.



Also, we can see that the months of September to December are more hectic than the first 8 months.
January to August has consistent spending of around 4k whereas September to December, the average spending increases significantly.

In [173]:
jan_to_aug_spend = sorted_ecom_df[(sorted_ecom_df["Date"] > "Jan, 2011") & 
                                  (sorted_ecom_df["Date"] < "Sep, 2011")].agg({
                                      "MoneySpent": pd.Series.mean
                                      })["MoneySpent"]
                                  
sep_to_dec_spend = sorted_ecom_df[(sorted_ecom_df["Date"] >= "Sep, 2011") & 
                                  (sorted_ecom_df["Date"] <= "Dec, 2011")].agg({
                                      "MoneySpent": pd.Series.mean
                                      })["MoneySpent"]
                                  
print(f"January to August average spending: {round(jan_to_aug_spend, 2)}")
print(f"September to December average spending: {round(sep_to_dec_spend, 2)}")
print(f"It equates to nearly {round(sep_to_dec_spend/jan_to_aug_spend, 1)}x more.")

January to August average spending: 4143.46
September to December average spending: 7274.0
It equates to nearly 1.8x more.


We can see the increase in spending is almost 1.8x. Hence, more marketing budget should be allotted to advertising products from September to December.


### Investigating Top Products Overall

In [229]:
top_spends = eu_ecom_df.groupby("Description").agg({
    "Quantity": pd.Series.sum,
    "MoneySpent": pd.Series.sum})
top_spends.reset_index(inplace=True)
top_spends.sort_values("Quantity", ascending=False, inplace=True)

In [228]:
most_popular_items = px.bar(top_spends[:20], y="Description", x="Quantity")
most_popular_items.show()

Most popular product overall is the `WORLD WAR 2 GLIDERS ASSTD DESIGNS` with 8902 items sold. <br>
Most spent on product is the `REGENCY CAKESTAND 3 TIER` with $23500.14 in revenue.



## Most Purchased Items in the Holiday Season

In [253]:
items_df = eu_ecom_df.groupby(["Date", "Description"]).agg({"Quantity": pd.Series.sum,
                                                 "UnitPrice": pd.Series.sum,
                                                 "MoneySpent": pd.Series.sum})
items_df.sort_values(["Date", "Quantity"], ascending=[True,False], inplace=True)
items_df.reset_index(inplace=True)
items_df.head(20)

,Date,Description,Quantity,UnitPrice,MoneySpent
0,2010-12-01,CHILLI LIGHTS,224,13.02,875.04
1,2010-12-01,WOODEN OWLS LIGHT GARLAND,192,3.37,647.04
2,2010-12-01,STRAWBERRY CERAMIC TRINKET BOX,156,2.50,195.00
3,2010-12-01,WHITE HANGING HEART T-LIGHT HOLDER,143,11.00,368.25
4,2010-12-01,HAND WARMER SCOTTY DOG DESIGN,136,8.15,261.60
5,2010-12-01,JUMBO BAG BAROQUE BLACK WHITE,100,1.65,165.00
6,2010-12-01,PINK HEART SHAPE EGG FRYING PAN,96,1.25,120.00
7,2010-12-01,RED HARMONICA IN BOX,96,3.75,120.00
8,2010-12-01,ANTIQUE SILVER TEA GLASS ENGRAVED,72,1.06,76.32
9,2010-12-01,CHARLOTTE BAG SUKI DESIGN,60,1.70,51.00


In [251]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67391 entries, 0 to 67390
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         67391 non-null  datetime64[ns]
 1   Description  67391 non-null  object        
 2   Quantity     67391 non-null  int64         
 3   UnitPrice    67391 non-null  float64       
 4   MoneySpent   67391 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 2.6+ MB


Filter out for holiday season

In [178]:
holiday_list = df_merged["Date"]
holiday_list.head(20)

7    2011-11-23
1    2011-10-10
9    2011-11-11
0    2011-07-04
3    2011-09-05
6    2011-11-24
10   2011-02-21
8    2011-02-14
2    2011-06-19
4    2011-09-04
5    2011-01-17
Name: Date, dtype: datetime64[ns]

Manually adding Christmas data as it is not available.

In [179]:
holiday_list.loc[len(holiday_list)] = "2010-12-25"
holiday_list

<ipython-input-179-8790538eb0cd>:1: FutureWarning:

Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})



7    2011-11-23
1    2011-10-10
9    2011-11-11
0    2011-07-04
3    2011-09-05
6    2011-11-24
10   2011-02-21
8    2011-02-14
2    2011-06-19
4    2011-09-04
5    2011-01-17
11   2010-12-25
Name: Date, dtype: datetime64[ns]

Filtering out data that does not have holidays near them.

In [180]:
items_hol = items_df[items_df["Date"].dt.month.isin(holiday_list.dt.month)]
items_hol.head()

,Date,Description,Quantity,UnitPrice,MoneySpent
52,2010-12-01,CHILLI LIGHTS,224,13.02,2916.48
282,2010-12-01,WOODEN OWLS LIGHT GARLAND,192,3.37,647.04
258,2010-12-01,STRAWBERRY CERAMIC TRINKET BOX,156,2.50,390.00
274,2010-12-01,WHITE HANGING HEART T-LIGHT HOLDER,143,11.00,1573.00
112,2010-12-01,HAND WARMER SCOTTY DOG DESIGN,136,8.15,1108.40


## Holidays vs Items

In [182]:
max_selling = items_hol.groupby([items_hol.Date.dt.month, items_hol.Description]).agg({"Quantity": pd.Series.sum})
max_selling.reset_index(inplace=True)
max_selling.sort_values(["Date", "Quantity"], ascending=[True, False], inplace=True)
max_selling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13928 entries, 412 to 12165
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         13928 non-null  int64 
 1   Description  13928 non-null  object
 2   Quantity     13928 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 435.2+ KB


Selecting top 10 items from every holiday month to track their selling performance throughout the year.

In [183]:
top_selling_items = pd.DataFrame(columns=max_selling.columns)

for date in range(1, 13):
  top_selling_items = top_selling_items.append(
      max_selling[max_selling["Date"] == date][:10], 
      ignore_index=False, verify_integrity=False, sort=None)

top_selling_items.describe()

,Date,Description,Quantity
count,80,80,80
unique,8,62,78
top,1,ASSORTED COLOUR BIRD ORNAMENT,535
freq,10,4,2


In [189]:
# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}], 
         [{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=4, cols=2, specs=specs, 
                    subplot_titles=["January", "February", "June", "July", 
                                    "September", "October", "November", "December"])

# Define pie charts
fig.add_trace(go.Pie(labels=top_selling_items["Description"][:10], 
                     values=top_selling_items["Quantity"][:10], name='January',
                     marker_colors=px.colors.sequential.ice), 1, 1)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][10:20], 
                     values=top_selling_items["Quantity"][10:20], name='February',
                     marker_colors=px.colors.sequential.ice), 1, 2)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][20:30], 
                     values=top_selling_items["Quantity"][20:30], name='June',
                     marker_colors=px.colors.sequential.ice), 2, 1)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][30:40], 
                     values=top_selling_items["Quantity"][30:40], name='July',
                     marker_colors=px.colors.sequential.ice), 2, 2)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][40:50],
                     values=top_selling_items["Quantity"][40:50], name='September',
                     marker_colors=px.colors.sequential.ice), 3, 1)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][50:60], 
                     values=top_selling_items["Quantity"][50:60], name='October',
                     marker_colors=px.colors.sequential.ice), 3, 2)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][60:70], 
                     values=top_selling_items["Quantity"][60:70], name='November',
                     marker_colors=px.colors.sequential.ice), 4, 1)

fig.add_trace(go.Pie(labels=top_selling_items["Description"][70:80], 
                     values=top_selling_items["Quantity"][70:80], name='December',
                     marker_colors=px.colors.sequential.ice), 4, 2)

# Tune layout, figure size and hover info
fig.update_traces(hoverinfo='label+percent+name', textinfo='none')

fig.update_layout(autosize=False, width=1000, height=2000,
                  showlegend=False, 
                  titlefont=dict(size=30, color='black'),
                  title={
                    'text': "<b>Number of Items Bought During the Holidays</b>",
                    'y':0.99,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                   ) 

fig = go.Figure(fig)
fig.show()

The pie charts show us the top 10 most purchased items in the holiday months. Hover over them to find out the names and quantites of these items. <br>
**Insights:**


1.   Christmas items start gaining popularity as early as September.
2.   Other decorative items maintain their popularity throughout the year.


<br> <br>



### Christmas Items Throughout the Year
We will now track the popularity of items that have christmas in their names.




In [297]:
# Building the DF for Christmas items
xmas_items = eu_ecom_df[eu_ecom_df["Description"].str.
                        contains("CHRISTMAS|xmas", 
                                 case=False, 
                                 regex=True)].groupby("Date").agg({
                                     "Quantity": pd.Series.sum,
                                     "MoneySpent": pd.Series.sum
                                     })
xmas_items.reset_index(inplace=True)
xmas_items.sort_values(["Date", "Quantity"], ascending=[True, False], 
                       inplace=True)

# Building the DF for Valentine items
val_items = eu_ecom_df[eu_ecom_df["Description"].str.
                       contains("heart|love|valentine", 
                                case=False, 
                                regex=True)].groupby(["Date"]).agg({
                                    "Quantity": pd.Series.sum,
                                    "MoneySpent": pd.Series.sum
                                    })
val_items.reset_index(inplace=True)
val_items.sort_values(["Date", "Quantity"], ascending=[True, False], 
                      inplace=True)

# Plotting the comparison chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=xmas_items["Date"], y=xmas_items["MoneySpent"],
                    mode='lines', fill='tozeroy', name='Christmas Items',
                    opacity=1))
fig.add_trace(go.Scatter(x=val_items["Date"], y=val_items["MoneySpent"],
                    mode='lines', fill='tozeroy', name='Valentine Items'))

fig.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                          text = "Comparing Item Sales for Valentine's Vs Christmas",
                          y = 0.95,
                          x = 0.5,
                          xanchor =  'center',
                          yanchor = 'top'
                          ),
                      legend=dict(
                          orientation = "h", 
                          yanchor = "bottom", 
                          y = 1.02, 
                          xanchor = "right",
                          x = 1))

fig.show()

In [242]:
xmas_vs_valentine = go.Figure()
xmas_vs_valentine.add_bar(x=["Christmas", "Valentine"], 
                          y=[xmas_items["MoneySpent"].sum(), 
                             val_items["MoneySpent"].sum()])
xmas_vs_valentine.update_layout(autosize=False, width=800, height=500,
                                title=dict(
                                  text = "Total Money Spent on Christmas Items vs Valentine's Items Throughout the Year",
                                  y = 0.95,
                                  x = 0.5,
                                  xanchor =  'center',
                                  yanchor = 'top'),
                                yaxis=dict(title_text="Total Money Spent"))
xmas_vs_valentine.show()

We can see that the there is a steady rise in Christmas items from mid-August and keep increasing till peak Christmas season. <br> <br>

We expected items like Valentine's Day gifts to be popular mostly around the Valentines Season but looking at the data, that doesn't seem true. They are not extremely popular but keep selling whole year round.

### Other Holidays

In [273]:
# Columbus DF
columbus_df = items_df[(items_df["Date"] > "2011-10-05") & (items_df["Date"] < "2011-10-11")].groupby("Description").agg({
    "MoneySpent": pd.Series.sum
})
columbus_df.sort_values("MoneySpent", ascending=False, inplace=True)
columbus_df.reset_index(inplace=True)


# 4th of July DF
july_4_df = items_df[(items_df["Date"] > "2011-06-29") & (items_df["Date"] <= "2011-07-04")].groupby("Description").agg({
    "MoneySpent": pd.Series.sum
})
july_4_df.sort_values("MoneySpent", ascending=False, inplace=True)
july_4_df.reset_index(inplace=True)


# Labor Day DF
labor_day_df = items_df[(items_df["Date"] >= "2011-09-01") & (items_df["Date"] <= "2011-09-05")].groupby("Description").agg({
    "MoneySpent": pd.Series.sum
})
labor_day_df.sort_values("MoneySpent", ascending=False, inplace=True)
labor_day_df.reset_index(inplace=True)


# Thanksgiving DF
thanksgiving_df = items_df[(items_df["Date"] >= "2011-11-20") & (items_df["Date"] <= "2011-11-24")].groupby("Description").agg({
    "MoneySpent": pd.Series.sum
})
thanksgiving_df.sort_values("MoneySpent", ascending=False, inplace=True)
thanksgiving_df.reset_index(inplace=True)

In [311]:
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]

most_popular_items = make_subplots(rows=1, cols=4, specs=specs, 
                    subplot_titles=["Columbus Day", "4th of July", "Labor Day", "Thanksgiving"])

most_popular_items.add_trace(go.Pie(labels=columbus_df["Description"][:5], 
                     values=columbus_df["MoneySpent"][:5], name='Columbus Day',
                     marker_colors=px.colors.sequential.ice,
                     hoverinfo="label"), 1, 1)

most_popular_items.add_trace(go.Pie(labels=july_4_df["Description"][:5], 
                     values=july_4_df["MoneySpent"][:10], name='4th of July',
                     marker_colors=px.colors.sequential.ice,
                     hoverinfo="label"), 1, 2)

most_popular_items.add_trace(go.Pie(labels=labor_day_df["Description"][:5], 
                     values=labor_day_df["MoneySpent"][:5], name='Labor Day',
                     marker_colors=px.colors.sequential.ice,
                     hoverinfo="label"), 1, 3)

most_popular_items.add_trace(go.Pie(labels=thanksgiving_df["Description"][:5], 
                     values=thanksgiving_df["MoneySpent"][:5], name='Thanksgiving',
                     marker_colors=px.colors.sequential.ice,
                     hoverinfo="label"), 1, 4)

most_popular_items.update_layout(titlefont=dict(size=20, color='black'),
                      title=dict(
                          text = "Comparing Most Selling Items During Holidays",
                          y = 0.95,
                          x = 0.5,
                          xanchor = 'center',
                          yanchor = 'top'
                          ),
                      showlegend=False)

most_popular_items.show()